In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import xarray as xr
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

from keras.models import load_model, Model, Sequential
from keras.layers import AveragePooling2D, MaxPooling2D, UpSampling2D,Conv2DTranspose, Input, SimpleRNN, LSTM, Dense,Dropout, Activation , Flatten, ConvLSTM2D, Conv2D, Reshape, TimeDistributed, BatchNormalization
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K
from keras.utils.vis_utils import plot_model
from tensorflow.keras import regularizers
import keras
import tensorflow as tf


In [2]:
#Ubicación de los archivos

path = r"/Users/javier/Documents/Data/"   
path2 = r"/Users/javier/Downloads/"


#dswind= xr.open_dataset(path + 'download(2000-2009).nc') # set de presión

names = ['download(1979-1988).nc',
         'download(1989-1999).nc',
         'download(2000-2009).nc',
         ]


files = [path + name for name in names]

dswind= xr.open_mfdataset(files)



df0 = pd.read_csv(path2 + 'NCEP_Spectra2.csv',index_col='Unnamed: 0')  #Sólo los parámetros de oleaje, no es el espectro completo.






Fechas = df0[Desde: Hasta].index

NameError: name 'Desde' is not defined

In [3]:
'''
Hace un avarage pooling y luego un flatten si se le pide.

'''

def resize(Wind,i,flat=False):
    
    Wind = Wind.reshape(Wind.shape[0], Wind.shape[1], Wind.shape[2],1)

    Average_pool = AveragePooling2D(pool_size = int(10*i), strides= int(10*i)) #el poolsize y stride, están pensados para las futuras iteraciones

    model = Sequential([Average_pool]) 

    output = model.predict(Wind)    
    
    Wind = np.squeeze(output) 
    
    
    if flat:
        
        Wind = Wind.reshape(Wind.shape[0],-1)
    
    
    return Wind


def gradiente(X):
    
    grad = np.zeros(X.shape)
    
    for i in range(1,grad.shape[1]-1):
        
        for j in range(1,grad.shape[2]-1):
            
            grad[:,i,j] = np.sqrt((X[:,i-1,j]-X[:,i+1,j])**2 + (X[:,i,j-1]-X[:,i,j+1])**2) #magnitud de gradiente aunque le falta ser dividida por el diferencial de distancia entre puntos, pero al ser constante para todos los valores no debería afectar.
    
    return grad


def flatten(Var):
    
    return Var.reshape(Var.shape[0], Var.shape[1]*Var.shape[2])


def preparar_x(dswind, lotes = 20000, pooling = 1, flat = False):

    '''
    Función para poder crear un arreglo de datos que contenga todos los dato del dataserie,
    
    Es relevante, ya que se debe hacer por lotes para no exceder la capacidad de la RAM.
    
    lotes: int, corresponde al tamaño de cada lote a procesar mediante un avarage pooling
    
    pooling: int, densidad del avarage pooling.
    
    return, arreglo X correspondiente a la base de datos dswind completa con dimensión reducida

    '''


    Largototal = dswind.data.shape[0]

    j = lotes

    X = resize(np.array(dswind.isel(time = slice(0,j)).data), pooling)

    
    for i in range(lotes*2,Largototal,lotes):
        
        windpart = resize(np.array(dswind.isel(time = slice(j,i)).data), pooling)
        
        X = np.append(X,windpart, axis = 0)
        
        j = i
        
        print('Datos: ',i,'/',Largototal)
        
    windpart = resize(np.array(dswind.isel(time = slice(j,Largototal)).data), pooling)

    X = np.append(X,windpart, axis = 0)
    
    if flat:
        
        X = X.reshape(X.shape[0],X.shape[1]*X.shape[2])
    
    return X



def dividir(Var, step):

    return Var[:(Var.shape[0]//step)*step].reshape(Var.shape[0]//step, step, Var.shape[1])


def dividir2(Var1, Var2, timestep, desfase):
    Var1_d = []

    for t in range(timestep+desfase, Var1.shape[0],timestep):
        
        Var1_d.append(Var1[t - timestep -desfase:t])
        
    Var1_d = np.array(Var1_d)

    Var2_d = []
    
    for t in range(timestep+desfase, Var2.shape[0],timestep):
            
        Var2_d.append(Var2[t - timestep:t])
        
    Var2_d = np.array(Var2_d)    
     
    return Var1_d, Var2_d


def dividir3(X,Y, step):
    
    shape = X.shape
    X = X.reshape(shape[0],shape[1],shape[2],1)
    
    x = np.zeros([shape[0],shape[1],shape[2],step])

    for i in range(step,shape[0]):
        
        for j in range(step):
            
            x[i,:,:,j] =  X[i-j,:,:,0]
            

    return x[step:], Y[step:]

def get_set(X, Y, timestep, traslape):
    
    scx2 = MinMaxScaler([0,1])
    
    X_div, Y_div =  dividir2(X,scx2.fit_transform(Y), timestep, traslape)

    #X_div, Y_div =  X, scx2.fit_transform(Y)


    
    m = X_div.shape[0]

    x_train, x_test = X_div[:int(m*0.8)], X_div[int(m*0.8):]

    y_train, y_test = Y_div[:int(m*0.8)], Y_div[int(m*0.8):]

    print('X_train shape: ', x_train.shape)
    print('Y_train shape: ', y_train.shape)

    return x_train, x_test, y_train, y_test, scx2


def get_set2(X, Y,  traslape):
    
    scx2 = MinMaxScaler([0,1])
    
    #X_div, Y_div =  dividir2(X,scx2.fit_transform(Y), timestep, traslape)

    X_div, Y_div =  dividir3(X, scx2.fit_transform(Y),traslape)


    
    m = X_div.shape[0]

    x_train, x_test = X_div[:int(m*0.8)], X_div[int(m*0.8):]

    y_train, y_test = Y_div[:int(m*0.8)], Y_div[int(m*0.8):]

    print('X_train shape: ', x_train.shape)
    print('Y_train shape: ', y_train.shape)

    return x_train, x_test, y_train, y_test, scx2

def preparar(X):
    
    shape = X.shape
    
    x = np.zeros([shape[0],shape[1],shape[2],3])

    for i in range(shape[0]):
        
        for j in range(3):
            
            x[i,:,:,j] =  X[i,:,:,0]

            
    print('X shape: ', x.shape)


    return x 

def normalizar(x):
    
    return (x-x.min())/(x.max() -x.min())



def juntar(Var):
    
    return Var.reshape(Var.shape[0]*Var.shape[1],Var.shape[2])

In [4]:
from tensorflow.keras.applications import InceptionV3, ResNet50, Xception,InceptionResNetV2, ResNet152V2, VGG16

input_tensor = Input(shape=(140, 140, 3))

model =  ResNet50(input_tensor=input_tensor , weights = 'imagenet')

#model= Model(inputs=model.input)#, outputs=model.get_layer('avg_pool').output)


In [5]:

model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 140, 140, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 146, 146, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 70, 70, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 70, 70, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [23]:
Desde = '1979-01-01 03:00:00'

Hasta = '1989-12-30 21:00:00'

Presion = dswind.msl.sel(time=slice(Desde,  Hasta),longitude=slice(-140,-70),latitude=slice(0,-70))

Pres = preparar_x(Presion,5000,0.2)  

Pres_n = normalizar(Pres)

X = np.expand_dims(Pres_n, -1).astype("float32")

X_3  = preparar(X)

x_features = model.predict(X_3)

np.save('x_features(1979-1989).npy', x_features)

print('se guardó 1979-1989')

Desde = '1989-01-01 03:00:00'

Hasta = '1999-12-30 21:00:00'

Presion = dswind.msl.sel(time=slice(Desde,  Hasta),longitude=slice(-140,-70),latitude=slice(0,-70))

Pres = preparar_x(Presion,5000,0.2)  

Pres_n = normalizar(Pres)

X = np.expand_dims(Pres_n, -1).astype("float32")

X_3  = preparar(X)

x_features = model.predict(X_3)

np.save('x_features(1989-1999).npy', x_features)

print('se guardó 1989-1999')

Desde = '1989-01-01 03:00:00'

Hasta = '1999-12-30 21:00:00'

Presion = dswind.msl.sel(time=slice(Desde,  Hasta),longitude=slice(-140,-70),latitude=slice(0,-70))

Pres = preparar_x(Presion,5000,0.2)  

Pres_n = normalizar(Pres)

X = np.expand_dims(Pres_n, -1).astype("float32")

X_3  = preparar(X)

x_features = model.predict(X_3)

np.save('x_features(1989-1999).npy', x_features)

print('se guardó 1989-1999')

Datos:  10000 / 32135
Datos:  15000 / 32135
Datos:  20000 / 32135
Datos:  25000 / 32135
Datos:  30000 / 32135
X shape:  (32135, 140, 140, 3)
se guardó 1979-1989
Datos:  10000 / 32127
Datos:  15000 / 32127
Datos:  20000 / 32127
Datos:  25000 / 32127
Datos:  30000 / 32127
X shape:  (32127, 140, 140, 3)
se guardó 1989-1999


In [6]:
Desde = '1999-01-01 03:00:00'

Hasta = '2009-12-30 21:00:00'

Presion = dswind.msl.sel(time=slice(Desde,  Hasta),longitude=slice(-140,-70),latitude=slice(0,-70))

Pres = preparar_x(Presion,5000,0.2)  

Pres_n = normalizar(Pres)

X = np.expand_dims(Pres_n, -1).astype("float32")

X_3  = preparar(X)

x_features = model.predict(X_3)

#np.save('x_features(1999-2009).npy', x_features)

print('se guardó 1999-2009')

Datos:  10000 / 32135
Datos:  15000 / 32135
Datos:  20000 / 32135
Datos:  25000 / 32135
Datos:  30000 / 32135
X shape:  (32135, 140, 140, 3)
se guardó 1999-2009


In [145]:
Pres_n = normalizar(Pres)

X = np.expand_dims(Pres_n, -1).astype("float32")

In [146]:
X = resize(X,1,True)

In [147]:
X.shape

(32127, 196)

In [22]:
np.save('x_features(1999-2009).npy', x_features)

In [85]:
Hm0 = np.array(df0[Desde: Hasta]['Hm0']).reshape(-1,1)
Tp = np.array(df0[Desde: Hasta]['Tp']).reshape(-1,1)
MD = np.array(df0[Desde: Hasta]['Mean Dir']).reshape(-1,1)
DSD =  np.array(df0[Desde: Hasta]['DSD']).reshape(-1,1)

In [150]:
Y = normalizar(Tp)

In [11]:
x_features2, Y = dividir2(x_features, Y, timestep =1 , desfase=60)

In [151]:
X2, Y = dividir2(X, Y, timestep =1 , desfase=60)

In [152]:
Y.shape

(32066, 1, 1)

In [94]:
X2.shape

(32066, 61, 49)

In [153]:
Y= Y.reshape(32066, 1)

In [103]:
X2.shape

(32066, 2989)

In [154]:
X2 = X2.reshape(32066, 61* 196)

In [16]:
x_features2 = x_features2.reshape(32074, 61*1000)

In [155]:
x_train_feat = X2[:27000]

x_test_feat = X2[27000:]

y_train_f = Y[:27000]

y_test_f = Y[27000:]

In [156]:
from sklearn import linear_model

clf_t = linear_model.Ridge(alpha=.5,normalize=True)



In [157]:
clf_t = clf_t.fit(x_train_feat,y_train_f)

In [108]:
y_pred_t = clf_t.predict(x_train_feat)

np.round(np.corrcoef(y_train_f.T,y_pred_t.T)[0,1],3)

0.655

In [158]:
y_pred_t = clf_t.predict(x_test_feat)

np.round(np.corrcoef(y_test_f.T,y_pred_t.T)[0,1],3)

0.333

In [ ]:
y_

In [38]:
y_pred_te = clf_t.predict(x_test_feat)

np.round(np.corrcoef(y_test_f[41:].T,y_pred_te.T)[0,1],3)

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 5033 and the array at index 1 has size 5074